# TP5 MPI Calcul Stencil avec l'environnement de programmation EasyPAP
## M1 informatique, Université d'Orléans 2021/2022

L'objectif de ce TP est d'utiliser l'environnement [EasyPAP](https://gforgeron.gitlab.io/easypap/) pour mettre en place une première simulation du *jeu de la vie* de J. Conway et d'utiliser les outils de visualisation des traces EasyPAP pour observer son fonctionnement.

## 1. L'exemple *spin* 

Pour programmer en MPI avec EasyPAP à partir d'un noyau (*kernel*) il faut définir une variante *compute* adaptée à MPI. Il va falloir également initialiser l'environnement MPI qui permettra en particulier de définir les identifiants des processus et le nombre de processus de l'exécution parallèle.

### Fonction d'initialisation de l'environnement MPI

Il faut définir une fonction d'initialisation qui sera exécutée par EasyPAP et à partir de variables globales cette fonction permettra également de définir différentes valeurs comme l'identifiant du processus. Le nom de cette fonction dépend du kernel. Par exemple pour *spin* on pourra utiliser le code ci-dessous.

~~~C
static int mpi_rank = -1;
static int mpi_size = -1;

void spin_init_mpi(void)
{
  easypap_check_mpi();
  MPI_Comm_rank(MPI_COMM_WORLD, &mpi_rank);
  MPI_Comm_size(MPI_COMM_WORLD, &mpi_size);
}
~~~

### Variant du kernel

Pour l'exemple spin, on souhaite paralléliser le calcul des nouvelles images en répartissant le calcul des pixels sur les différents processus. 


~~~C
unsigned spin_compute_mpi(unsigned nb_iter)
{
  for (unsigned it=1; it<=nb_iter; it++) {
    do_tile(?,?,?,?,0);
    rotate();
  }
  // Ici le rassemblement des données pour l'affichage
  return 0;
}
~~~

### ⚠️ Attention

EasyPAP fournit une variable prédéfinie *cur\_img* qui permet de construire une image et de la manipuler. *cur\_img* est en fait une macro qui permet d'accéder aux pixels d'une image qui est stockée dans la variable prédéfinie *image*. Ainsi pour modifier les pixels vous pouvez utiliser *cur\_img* mais pour les routines de communication MPI, l'argument à donner c'est *image* avec le type MPI_INT. De même pour *next\_img* et *alt_image* avec la fonction *swap\_images()* prédéfinie pour échanger les 2 images dans un calcul itératif par exemple. Pour le premier exemple spin seul *cur\_img* et *image* sont utilisés.

### ⚠️ Attention 2

EasyPAP lors de l'initialisation de MPI initialise également une image complète sur tous les processus de l'exécution parallèle. Ainsi *image* sera de taille `DIM x DIM` sur tous les processus. Si on utilise l'option `--load-image` ou `-l` chaque processus disposera d'un tableau *image* initialisé avec l'image donnée en argument.

### Exécution du kernel, variant MPI

L'exécution du kernel en version MPI demande un argument supplémentaire pour indiquer le mode de lancement MPI qu'on souhaite utiliser : 

~~~sh
$ ./run -k spin -v mpi --mpirun "-np 2"
~~~

~~~sh
$ ./run -k spin -v mpi --mpirun "-np 4" --debug-flags M
~~~

~~~sh
$ ./run -k spin -v mpi --mpirun "-np 4" --trace --no-display -i 10 --thumbnails
~~~

Pour visualiser les traces il faut utiliser la commande ci-dessous où xx est à remplacer par le numéro du processus 
qui vous intéresse. On ne peut utiliser *view* qu'avec au plus 2 arguments. La navigation dans les traces se fait avec des commandes clavier/souris (cf la manuel [Getting Started](https://gforgeron.gitlab.io/easypap/doc/Getting_Started.pdf)). Par exemple pour aligner les données des deux traces vous pouvez *presser le bouton a*.

~~~sh
$ ./view traces/data/ezv_trace_current.xx.evt traces/data/ezv_trace_current_xx.evt
~~~

## 2. Jeu de la vie

Récupérez sur Celene l'archive `src.tgz` contenant un nouveau noyau `gameimage` et un dossier `images`. Ce noyau contient une première version (`-v seq`) qui met en œuvre le jeu de la vie. Vous devez copier le fichier `gameimage.c` dans le répertoire `kernel/c/` et vous pouvez également copier les images dans le répertoire `images/` d'EasyPAP.

Le point de départ de ce jeu de la vie est une image qu'il faut charger au lancement du kernel.

Après avoir compilé vous pourrez tester la version séquentielle avec 

~~~sh
$ ./run -k gameimage -v seq -l images/im_64.png
~~~

## Références

 1. [Spécification MPI](https://www.mpi-forum.org/docs/)
 1. [Documentation officielle d'EasyPAP](https://gforgeron.gitlab.io/easypap/doc/Getting_Started.pdf)